In [1]:
# Importing required libraries
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns 


/var/folders/4j/wqkxszcs5jl9436sl4hsmprh0000gn/T/ipykernel_59212/2359636322.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Loading Dataset
import csv
# Read the 3 different csv files 
books = pd.read_csv('/Users/rabin/College_Project_Seven/Books.csv', low_memory=False)
ratings = pd.read_csv('/Users/rabin/College_Project_Seven/Ratings.csv', encoding='latin-1', delimiter=';')
users = pd.read_csv('/Users/rabin/College_Project_Seven/Users.csv', encoding='latin-1', delimiter=';')


In [3]:
books.shape

(271360, 8)

In [4]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [5]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

Rename of Books

In [6]:
books.rename(columns={
    "Book-Title": "title",
    "Year-Of-Publication" : "year",
    "Book-Author" : "author",
    "Publisher" : "publisher",
    "Image-URL-L": "img_url"},
             inplace=True)

In [7]:
books.head()

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [8]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
ratings.rename(columns={
    "User-ID": "user_id",
    "Book-Rating": "rating"},
             inplace=True)

In [10]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [11]:
ratings['user_id'].unique().shape

(105283,)

In [12]:
x = ratings['user_id'].value_counts() >200

In [13]:
x[x].shape

(899,)

In [14]:
y = x[x].index

In [15]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [16]:
ratings = ratings[ratings['user_id'].isin(y)]

In [17]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [18]:
ratings.shape

(526356, 3)

In [19]:
ratings_with_books = ratings.merge(books, on= "ISBN")

In [20]:
ratings_with_books


,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...
...,...,...,...,...,...,...,...,...
487666,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...
487667,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...
487668,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...
487669,275970,4770019572,0,Musashi,Eiji Yoshikawa,1995,Kodansha International (JPN),http://images.amazon.com/images/P/4770019572.0...


In [21]:
ratings_with_books.shape

(487671, 8)

In [22]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [23]:
num_rating.shape

(160269, 2)

In [24]:
num_rating.rename(columns={"rating": "num_of_rating"},inplace = True)

In [25]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [26]:
final_rating = ratings_with_books.merge(num_rating, on='title')

In [27]:
final_rating.head(3)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1


In [28]:

final_rating.shape


(487671, 9)

In [29]:
# Display books with rating more than 50
final_rating = final_rating[final_rating['num_of_rating']>=50]


In [30]:
final_rating.sample(7)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
78714,38023,055321313X,10,Anne of Green Gables (Anne of Green Gables Nov...,L.M. MONTGOMERY,1982,Bantam Classics,http://images.amazon.com/images/P/055321313X.0...,90
137220,78783,0446612545,0,The Beach House,James Patterson,2003,Warner Books,http://images.amazon.com/images/P/0446612545.0...,142
453263,254465,014023313X,0,The Stone Diaries,Carol Shields,1995,Penguin Books,http://images.amazon.com/images/P/014023313X.0...,84
148186,87141,0385424728,8,The Chamber,John Grisham,1994,Doubleday Books,http://images.amazon.com/images/P/0385424728.0...,179
239847,137688,0440202043,0,Shell Seekers,Rosamunde Pilcher,1996,Dell,http://images.amazon.com/images/P/0440202043.0...,72
253865,147847,0671886665,5,A Cry In The Night,Mary Higgins Clark,1993,Pocket,http://images.amazon.com/images/P/0671886665.0...,50
440245,246617,0099760118,0,Beloved,Toni Morrison,0,Vintage Books,http://images.amazon.com/images/P/0099760118.0...,78


In [31]:
final_rating.shape

(61853, 9)

In [32]:
# remove duplicate 
final_rating.drop_duplicates(['title', 'user_id'],inplace=True)

In [33]:
final_rating.shape

(59850, 9)

In [34]:

book_pivot = final_rating.pivot_table(columns='user_id', index='title',values='rating') 

In [35]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [36]:
book_pivot.shape

(742, 888)

In [37]:
book_pivot.fillna(0, inplace=True)

In [38]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
import numpy as np

# Function to compute Euclidean distance between two vectors
def euclidean_distance(vector1, vector2):
    return np.sqrt(np.sum((vector1 - vector2)**2))

# Compute pairwise distances between rows using Euclidean distance
num_rows = book_pivot.shape[0]
distance_matrix = np.zeros((num_rows, num_rows))

for i in range(num_rows):
    for j in range(i+1, num_rows):
        distance = euclidean_distance(book_pivot.iloc[i].values, book_pivot.iloc[j].values)
        # Since the distance matrix is symmetric, we can assign the same value for distance_matrix[j, i]
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance

# distance_matrix[i, j] will give the Euclidean distance between row i and row j


In [40]:
distance_matrix

array([[ 0.        , 67.16397844, 66.41535967, ..., 53.38539126,
        50.84289528, 60.94259594],
       [67.16397844,  0.        , 63.30876717, ..., 58.83026432,
        53.4976635 , 61.22907806],
       [66.41535967, 63.30876717,  0.        , ..., 55.29014379,
        51.38093031, 58.94913061],
       ...,
       [53.38539126, 58.83026432, 55.29014379, ...,  0.        ,
        36.81032464, 50.15974482],
       [50.84289528, 53.4976635 , 51.38093031, ..., 36.81032464,
         0.        , 43.89760814],
       [60.94259594, 61.22907806, 58.94913061, ..., 50.15974482,
        43.89760814,  0.        ]])

In [66]:
# Find nearest neighbors for each row
nearest_indices = {}
k = 6
for i in range(num_rows):
    # Sort distances for the current row
    nearest_indices[i] = np.argsort(distance_matrix[i])[:k]

# nearest_indices[i] will give the indices of the k nearest neighbors for row i


In [69]:
distances = distance_matrix


In [72]:
# Let's say nearest_indices is already computed and contains the indices of nearest neighbors for each row
# Assuming nearest_indices is a dictionary where each key represents a row index and the value is an array of indices of nearest neighbors
# Let's say we want to print the titles of recommended books for each row

for i in range(len(nearest_indices)):
    print("Recommended books for:", book_pivot.index[i])
    for j in nearest_indices[i]:
        print("- ", book_pivot.index[j])
    print()  # Adding a blank line for better readability


Recommended books for: 1984
-  1984
-  No Safe Place
-  Long After Midnight
-  A Civil Action
-  Abduction
-  Master of the Game

Recommended books for: 1st to Die: A Novel
-  1st to Die: A Novel
-  Exclusive
-  The Cradle Will Fall
-  No Safe Place
-  Fine Things
-  The Face of Deception

Recommended books for: 2nd Chance
-  2nd Chance
-  The Next Accident
-  The Ghost
-  Exclusive
-  Last Man Standing
-  Unspeakable

Recommended books for: 4 Blondes
-  4 Blondes
-  No Safe Place
-  Pleading Guilty
-  Long After Midnight
-  Exclusive
-  Lake Wobegon days

Recommended books for: 84 Charing Cross Road
-  84 Charing Cross Road
-  Exclusive
-  Jacob Have I Loved
-  Alaska
-  No Safe Place
-  The Little Friend

Recommended books for: A Bend in the Road
-  A Bend in the Road
-  Exclusive
-  The Cradle Will Fall
-  No Safe Place
-  Family Album
-  Last Man Standing

Recommended books for: A Case of Need
-  A Case of Need
-  Exclusive
-  Jacob Have I Loved
-  Pleading Guilty
-  No Safe Place


In [73]:
book_pivot.index[3]

'4 Blondes'

In [74]:
book_pivot.index


Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [76]:
book_name = book_pivot.index

In [77]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distances, nearest_indices = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1), n_neighbors=6)

    for i in range(len(nearest_indices)):
        books = book_pivot.index[nearest_indices[i]]
        for j in books:
            print(j)


In [78]:

# Recommendation function
def recommend(book_name):
    try:
        if book_name in pt.index:
            # index fetch
            index = np.where(pt.index == book_name)[0][0]
            similar_items = sorted(list(enumerate(similarity_scores_manual[index])), key=lambda x: x[1], reverse=True)[1:5]
            
            data = []
            for i in similar_items:
                item = {}
                temp_df = books[books['Book-Title'] == pt.index[i[0]]]
                item['BookTitle'] = temp_df.drop_duplicates('Book-Title')['Book-Title'].values[0]
                #item['ISBN'] = temp_df.drop_duplicates('Book-Title')['ISBN'].values[0]  # Assuming ISBN is a column in your dataset
                #item['ImageURL'] = temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values[0]
                
                data.append(item)
            return data
        else:
            print(f"Book '{book_name}' not found in the dataset.")
            return None
    except Exception as e:
        logging.error(f"An error occurred: {str(e)}")
        return None




In [79]:
book_name ='Harry Potter and the Chamber of Secrets (Book 2)'
recommend(book_name)
              

NameError: name 'logging' is not defined